In [1]:
import ipywidgets as widgets

pdf_uploader = widgets.FileUpload(description="Original pdfs", accept=".pdf", multiple=True)
tag_uploader = widgets.FileUpload(description="Tag image/pdf", accept="image/*,.pdf", multiple=False)
output = widgets.Output()

In [2]:
@output.capture()
def write_pdf(content):
    for k, v in content.items():
        print("Loading {}...".format(k), end=" ")
        with open(k, "wb") as f:
            f.write(v["content"])
        print("done")
            
def write_original_pdf(counter):
    if counter.get("new") == 0:
        return
    write_pdf(pdf_uploader.value)
def write_tag_pdf(counter):
    if counter.get("new") == 0:
        return
    write_pdf(tag_uploader.value)
    
pdf_uploader.observe(write_original_pdf, names="_counter")
tag_uploader.observe(write_tag_pdf, names="_counter")

In [3]:
def clear(_):
    output.clear_output()
    pdf_uploader._counter = 0
    tag_uploader._counter = 0
    pdf_uploader.value.clear()
    tag_uploader.value.clear()

In [4]:
xpos = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description="$x$")
ypos = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description="$y$")
width = widgets.FloatSlider(value=0.7, min=0.0, max=1.0, step=0.01, description="width")

In [5]:
@output.capture()
def process(_):
    import subprocess

    if len(pdf_uploader.value) == 0:
        print("Missing file to tag!")
        return

    if len(tag_uploader.value) == 0:
        print("Missing tag!")
        return

    tag_name = list(tag_uploader.value.keys())[0]
    for pdf_name in pdf_uploader.value.keys():
        print("Processing {}...".format(pdf_name), end=" ")
        result = subprocess.run(
            [
                "./bin/generate_tag.sh",
                "--multi",
                pdf_name,
                "--tag",
                tag_name,
                "-x",
                str(xpos.value),
                "-y",
                str(ypos.value),
                "-w",
                str(width.value),
            ],
            capture_output=True,
            text=True,
        )
        if result.returncode != 0:
            print("failed!")
            print("Check the following messages to know what's going on:")
            print(result.stdout)
            print(result.stderr)
        else:
            print("done")

In [6]:
@output.capture()
def download(_):
    from IPython.display import HTML
    for pdf_name in pdf_uploader.value.keys():
        js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download','');
        a.setAttribute('href', '../files/bin/pdf/{pdf_name}');
        a.click()
    """
        with output:
            display(HTML(f"<script>{js_code}</script>"))

In [7]:
process_button = widgets.Button(description="Process", icon="check", button_style="success")
download_button = widgets.Button(description="Download", icon="download", button_style="warning")
clear_button = widgets.Button(description="Clear", icon="trash", button_style="danger")
clear_button.on_click(clear)
download_button.on_click(download)
process_button.on_click(process)

top = widgets.VBox(
    [
        widgets.HTML(value="Tag position/width in fraction of the page"),
        widgets.HBox([widgets.HTML(value="Select pdf(s) to tag"), pdf_uploader, xpos]),
        widgets.HBox([widgets.HTML(value="Select tag image"), tag_uploader, ypos]),
        width,
    ],
    layout=widgets.Layout(display="flex", align_items="flex-end"),
)

bottom = widgets.HBox([process_button, download_button, clear_button])

widgets.HBox(
    [
        widgets.HTML(value="<h1>Tag Application</h1>"),
        top,
        widgets.HTML(value="<p style='margin-bottom:1cm;'></p>"),
        bottom,
        widgets.VBox([widgets.HTML(value="<h2>Logs</h2>"), output]),
    ],
    layout=widgets.Layout(display="flex", flex_flow="column", align_items="center", width="100%"),
)